In [159]:
import pandas as pd
import numpy as np
from embedding import get_embedding

In [160]:
data = {'ESTIMATE': ["I think [AGENT] is a [ROLE]", "[AGENT] must be the [ROLE]", "[AGENT] seems like the [ROLE]", "I'm convinced that [AGENT] is a [ROLE]", "I think [AGENT] is a [ROLE], not a"],
                          'CO': ["I am a [ROLE]", "My role is the [ROLE]", "I must be the [ROLE]", "There are no other [ROLE] besides me"], 
                          'SUSPECT': ["I can't believe [AGENT]", "[AGENT] is suspicious", "I suspect [AGENT] is lying", "I don't trust [AGENT]", "[AGENT] cannot be trusted", "[AGENT], you are lying"], 
                          'VOTE': ["I will vote for [AGENT]", "Vote for [AGENT] tonight"], 
                          'DIVINATION': ["I will divine [AGENT] tonight", "I might investigate [AGENT]"], 
                          'DIVINED': ["I divined [AGENT] as a [TEAM]", "[AGENT] was divined as a [TEAM]", "[AGENT] is determined to be [TEAM]"], 
                          'AGREE': ["I agree with [AGENT]", "I agree with [AGENT]'s opinion", "What [AGENT] said is correct"], 
                          'NOT ESTIMATE': ["I don't think [AGENT] is a [ROLE]", "[AGENT] is not [ROLE]", "[AGENT] doesn't seem like the [ROLE]", "[AGENT] is lying about being a [ROLE]"],
                          'NOT CO': ["I am not a [ROLE]", "It can't be true that I am the [ROLE]"], 
                          'NOT SUSPECT': [ "I don't think [AGENT] is lying", "I don't think [AGENT] is suspicious", "I trust [AGENT]", '[AGENT] is right'], 
                          'NOT VOTE': ["I won't vote for [AGENT]", "I disagree with voting for [AGENT]"], 
                          'NOT AGREE': ["I disagree with [AGENT]",  "I disagree with [AGENT]'s opinion", "What [AGENT] said is wrong"], 
                          }

In [161]:
df = pd.DataFrame.from_dict(data, orient='index').T # これだとうまくいく
df

,ESTIMATE,CO,SUSPECT,VOTE,DIVINATION,DIVINED,AGREE,NOT ESTIMATE,NOT CO,NOT SUSPECT,NOT VOTE,NOT AGREE
0,I think [AGENT] is a [ROLE],I am a [ROLE],I can't believe [AGENT],I will vote for [AGENT],I will divine [AGENT] tonight,I divined [AGENT] as a [TEAM],I agree with [AGENT],I don't think [AGENT] is a [ROLE],I am not a [ROLE],I don't think [AGENT] is lying,I won't vote for [AGENT],I disagree with [AGENT]
1,[AGENT] must be the [ROLE],My role is the [ROLE],[AGENT] is suspicious,Vote for [AGENT] tonight,I might investigate [AGENT],[AGENT] was divined as a [TEAM],I agree with [AGENT]'s opinion,[AGENT] is not [ROLE],It can't be true that I am the [ROLE],I don't think [AGENT] is suspicious,I disagree with voting for [AGENT],I disagree with [AGENT]'s opinion
2,[AGENT] seems like the [ROLE],I must be the [ROLE],I suspect [AGENT] is lying,None,None,[AGENT] is determined to be [TEAM],What [AGENT] said is correct,[AGENT] doesn't seem like the [ROLE],None,I trust [AGENT],None,What [AGENT] said is wrong
3,I'm convinced that [AGENT] is a [ROLE],There are no other [ROLE] besides me,I don't trust [AGENT],None,None,None,None,[AGENT] is lying about being a [ROLE],None,[AGENT] is right,None,None
4,"I think [AGENT] is a [ROLE], not a",None,[AGENT] cannot be trusted,None,None,None,None,None,None,None,None,None
5,None,None,"[AGENT], you are lying",None,None,None,None,None,None,None,None,None


In [162]:
df.fillna(np.nan, inplace=True)
df

,ESTIMATE,CO,SUSPECT,VOTE,DIVINATION,DIVINED,AGREE,NOT ESTIMATE,NOT CO,NOT SUSPECT,NOT VOTE,NOT AGREE
0,I think [AGENT] is a [ROLE],I am a [ROLE],I can't believe [AGENT],I will vote for [AGENT],I will divine [AGENT] tonight,I divined [AGENT] as a [TEAM],I agree with [AGENT],I don't think [AGENT] is a [ROLE],I am not a [ROLE],I don't think [AGENT] is lying,I won't vote for [AGENT],I disagree with [AGENT]
1,[AGENT] must be the [ROLE],My role is the [ROLE],[AGENT] is suspicious,Vote for [AGENT] tonight,I might investigate [AGENT],[AGENT] was divined as a [TEAM],I agree with [AGENT]'s opinion,[AGENT] is not [ROLE],It can't be true that I am the [ROLE],I don't think [AGENT] is suspicious,I disagree with voting for [AGENT],I disagree with [AGENT]'s opinion
2,[AGENT] seems like the [ROLE],I must be the [ROLE],I suspect [AGENT] is lying,NaN,NaN,[AGENT] is determined to be [TEAM],What [AGENT] said is correct,[AGENT] doesn't seem like the [ROLE],NaN,I trust [AGENT],NaN,What [AGENT] said is wrong
3,I'm convinced that [AGENT] is a [ROLE],There are no other [ROLE] besides me,I don't trust [AGENT],NaN,NaN,NaN,NaN,[AGENT] is lying about being a [ROLE],NaN,[AGENT] is right,NaN,NaN
4,"I think [AGENT] is a [ROLE], not a",NaN,[AGENT] cannot be trusted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,"[AGENT], you are lying",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
df.columns

Index(['ESTIMATE', 'CO', 'SUSPECT', 'VOTE', 'DIVINATION', 'DIVINED', 'AGREE',
       'NOT ESTIMATE', 'NOT CO', 'NOT SUSPECT', 'NOT VOTE', 'NOT AGREE'],
      dtype='object')

In [164]:
data2 = {
    'S_ESTIMATE': [
        "[S] thinks [AGENT] is a [ROLE]", 
        "[S] believes [AGENT] must be the [ROLE]", 
        "[S] thinks [AGENT] seems like the [ROLE]", 
        "[S] is convinced that [AGENT] is a [ROLE]", 
        "[S] thinks [AGENT] is a [ROLE], not a"
        "[AGENT] is suspected by [S] as a [ROLE]"
    ],
    'S_CO': [
        "[S] claims to be a [ROLE]", 
        "[S] says their role is the [ROLE]", 
        "[S] claims they must be the [ROLE]", 
        "[S] states there are no other [ROLE] besides them"
    ],
    'S_SUSPECT': [
        "[S] can't believe [AGENT]", 
        "[S] finds [AGENT] suspicious", 
        "[S] suspects [AGENT] is lying", 
        "[S] doesn't trust [AGENT]", 
        "[S] says [A] cannot be trusted"
        "[AGENT] is suspected by [S]"
    ],
    'S_VOTE': [
        "[S] will vote for [AGENT]"
    ],
    'S_DIVINATION': [
        "[S] will divine [AGENT] tonight", 
        "[S] might investigate [AGENT]"
    ],
    'S_DIVINED': [
        "[S] divined [AGENT] as a [TEAM]", 
        "[S] determined [AGENT] to be [TEAM]"
    ],
    'S_AGREE': [
        "[S] agrees with [AGENT]", 
        "[S] agrees with [AGENT]'s opinion", 
        "[S] thinks what [AGENT] said is correct"
    ],
    'S_NOT_ESTIMATE': [
        "[S] doesn't think [AGENT] is a [ROLE]", 
        "[S] says [AGENT] is not [ROLE]", 
        "[S] doesn't think [AGENT] seems like the [ROLE]", 
        "[S] thinks [AGENT] is lying about being a [ROLE]"
    ],
    'S_NOT_CO': [
        "[S] claims not to be a [ROLE]", 
        "[S] says it can't be true that they are the [ROLE]",
        "[S] says he's not a [ROLE]"
    ],
    'S_NOT_SUSPECT': [
        "[S] doesn't think [AGENT] is lying", 
        "[S] trusts [AGENT]", 
        "[S] thinks [AGENT] is right"
    ],
    'S_NOT_VOTE': [
        "[S] won't vote for [AGENT]", 
        "[S] disagrees with voting for [AGENT]"
    ],
    'S_NOT_AGREE': [
        "[S] disagrees with [AGENT]",  
        "[S] disagrees with [AGENT]'s opinion", 
        "[S] thinks what [AGENT] said is wrong"
    ],
}


In [165]:
df2 = pd.DataFrame.from_dict(data2, orient='index').T # これだとうまくいく
df2.fillna(np.nan, inplace=True)
df2

,S_ESTIMATE,S_CO,S_SUSPECT,S_VOTE,S_DIVINATION,S_DIVINED,S_AGREE,S_NOT_ESTIMATE,S_NOT_CO,S_NOT_SUSPECT,S_NOT_VOTE,S_NOT_AGREE
0,[S] thinks [AGENT] is a [ROLE],[S] claims to be a [ROLE],[S] can't believe [AGENT],[S] will vote for [AGENT],[S] will divine [AGENT] tonight,[S] divined [AGENT] as a [TEAM],[S] agrees with [AGENT],[S] doesn't think [AGENT] is a [ROLE],[S] claims not to be a [ROLE],[S] doesn't think [AGENT] is lying,[S] won't vote for [AGENT],[S] disagrees with [AGENT]
1,[S] believes [AGENT] must be the [ROLE],[S] says their role is the [ROLE],[S] finds [AGENT] suspicious,NaN,[S] might investigate [AGENT],[S] determined [AGENT] to be [TEAM],[S] agrees with [AGENT]'s opinion,[S] says [AGENT] is not [ROLE],[S] says it can't be true that they are the [R...,[S] trusts [AGENT],[S] disagrees with voting for [AGENT],[S] disagrees with [AGENT]'s opinion
2,[S] thinks [AGENT] seems like the [ROLE],[S] claims they must be the [ROLE],[S] suspects [AGENT] is lying,NaN,NaN,NaN,[S] thinks what [AGENT] said is correct,[S] doesn't think [AGENT] seems like the [ROLE],[S] says he's not a [ROLE],[S] thinks [AGENT] is right,NaN,[S] thinks what [AGENT] said is wrong
3,[S] is convinced that [AGENT] is a [ROLE],[S] states there are no other [ROLE] besides them,[S] doesn't trust [AGENT],NaN,NaN,NaN,NaN,[S] thinks [AGENT] is lying about being a [ROLE],NaN,NaN,NaN,NaN
4,"[S] thinks [AGENT] is a [ROLE], not a[AGENT] i...",NaN,[S] says [A] cannot be trusted[AGENT] is suspe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [166]:
Df = pd.concat([df, df2], axis=1)

In [167]:
Df

,ESTIMATE,CO,SUSPECT,VOTE,DIVINATION,DIVINED,AGREE,NOT ESTIMATE,NOT CO,NOT SUSPECT,...,S_SUSPECT,S_VOTE,S_DIVINATION,S_DIVINED,S_AGREE,S_NOT_ESTIMATE,S_NOT_CO,S_NOT_SUSPECT,S_NOT_VOTE,S_NOT_AGREE
0,I think [AGENT] is a [ROLE],I am a [ROLE],I can't believe [AGENT],I will vote for [AGENT],I will divine [AGENT] tonight,I divined [AGENT] as a [TEAM],I agree with [AGENT],I don't think [AGENT] is a [ROLE],I am not a [ROLE],I don't think [AGENT] is lying,...,[S] can't believe [AGENT],[S] will vote for [AGENT],[S] will divine [AGENT] tonight,[S] divined [AGENT] as a [TEAM],[S] agrees with [AGENT],[S] doesn't think [AGENT] is a [ROLE],[S] claims not to be a [ROLE],[S] doesn't think [AGENT] is lying,[S] won't vote for [AGENT],[S] disagrees with [AGENT]
1,[AGENT] must be the [ROLE],My role is the [ROLE],[AGENT] is suspicious,Vote for [AGENT] tonight,I might investigate [AGENT],[AGENT] was divined as a [TEAM],I agree with [AGENT]'s opinion,[AGENT] is not [ROLE],It can't be true that I am the [ROLE],I don't think [AGENT] is suspicious,...,[S] finds [AGENT] suspicious,NaN,[S] might investigate [AGENT],[S] determined [AGENT] to be [TEAM],[S] agrees with [AGENT]'s opinion,[S] says [AGENT] is not [ROLE],[S] says it can't be true that they are the [R...,[S] trusts [AGENT],[S] disagrees with voting for [AGENT],[S] disagrees with [AGENT]'s opinion
2,[AGENT] seems like the [ROLE],I must be the [ROLE],I suspect [AGENT] is lying,NaN,NaN,[AGENT] is determined to be [TEAM],What [AGENT] said is correct,[AGENT] doesn't seem like the [ROLE],NaN,I trust [AGENT],...,[S] suspects [AGENT] is lying,NaN,NaN,NaN,[S] thinks what [AGENT] said is correct,[S] doesn't think [AGENT] seems like the [ROLE],[S] says he's not a [ROLE],[S] thinks [AGENT] is right,NaN,[S] thinks what [AGENT] said is wrong
3,I'm convinced that [AGENT] is a [ROLE],There are no other [ROLE] besides me,I don't trust [AGENT],NaN,NaN,NaN,NaN,[AGENT] is lying about being a [ROLE],NaN,[AGENT] is right,...,[S] doesn't trust [AGENT],NaN,NaN,NaN,NaN,[S] thinks [AGENT] is lying about being a [ROLE],NaN,NaN,NaN,NaN
4,"I think [AGENT] is a [ROLE], not a",NaN,[AGENT] cannot be trusted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,[S] says [A] cannot be trusted[AGENT] is suspe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,"[AGENT], you are lying",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
class MakeDataframe:
    def __init__(self, df):
        self.df = df
        self.datalist = None

    def make_data(self):
        """データフレームを受け取り、[AGENT]と[ROLE]と[TEAM]を全ての組み合わせで埋める"""
        Subjects = ["Agent", "Someone", "Everyone"]
        Agents = ["Agent[00]", "Agent[01]", "Agent[02]", "Agent[03]", "Agent[04]"]
        Roles = ["villeger", "seer", "werewolf", "possessed"]
        Teams = ["black", "white", "villager", "human", "werewolf"] # 精度のためにteamsに"VILLAGER", "WEREWOLF"も追加
        
        self.datalist = []
        for column in self.df.columns: 
            original_sentences = self.df[column].dropna().values
            for original_sentence in original_sentences:
                
                if "[S]" in original_sentence:
                    for s in Subjects:
                        
                        original_sentence_here = original_sentence.replace("[S]", s)
                        if "[AGENT]" in original_sentence_here and "[ROLE]" in original_sentence_here:
                            for x in Agents:
                                for y in Roles:
                                    ans = {} # 各sentenceの情報を入れる辞書
                                    ans["sentence"] = original_sentence_here.replace('[AGENT]', x).replace('[ROLE]', y)
                                    ans["Verb"] = None
                                    ans["Agent"] = x
                                    ans["Role"] = y.upper()
                                    ans["Team"] = None
                                    self.datalist.append(ans)
                        elif "[AGENT]" in original_sentence_here and "[TEAM]" in original_sentence_here:
                            for x in Agents:
                                for y in Teams:
                                    ans = {} # 各sentenceの情報を入れる辞書
                                    ans["sentence"] = original_sentence_here.replace('[AGENT]', x).replace('[TEAM]', y)
                                    ans["Verb"] = None
                                    ans["Agent"] = x
                                    ans["Role"] = None
                                    ans["Team"] = y.upper()
                                    self.datalist.append(ans)
                        elif "[AGENT]" in original_sentence_here:
                            for x in Agents:
                                ans = {} # 各sentenceの情報を入れる辞書
                                ans["sentence"] = original_sentence_here.replace('[AGENT]', x)
                                ans["Verb"] = None
                                ans["Agent"] = x
                                ans["Role"] = None
                                ans["Team"] = None
                                self.datalist.append(ans)
                        elif '[ROLE]' in original_sentence_here:
                            for y in Roles:
                                ans = {} # 各sentenceの情報を入れる辞書
                                ans["sentence"] = original_sentence_here.replace('[ROLE]', y)
                                ans["Verb"] = None
                                ans["Agent"] = None
                                ans["Role"] = y.upper()
                                ans["Team"] = None
                                self.datalist.append(ans)

                else:
                    if "[AGENT]" in original_sentence and "[ROLE]" in original_sentence:
                        for x in Agents:
                            for y in Roles:
                                ans = {} # 各sentenceの情報を入れる辞書
                                ans["sentence"] = original_sentence.replace('[AGENT]', x).replace('[ROLE]', y)
                                ans["Verb"] = column
                                ans["Agent"] = x
                                ans["Role"] = y.upper()
                                ans["Team"] = None
                                
                                self.datalist.append(ans)
                    elif "[AGENT]" in original_sentence and "[TEAM]" in original_sentence:
                        for x in Agents:
                            for y in Teams:
                                ans = {} # 各sentenceの情報を入れる辞書
                                ans["sentence"] = original_sentence.replace('[AGENT]', x).replace('[TEAM]', y)
                                ans["Verb"] = column
                                ans["Agent"] = x
                                ans["Role"] = None
                                ans["Team"] = y.upper()
                                self.datalist.append(ans)
                    elif "[AGENT]" in original_sentence:
                        for x in Agents:
                            ans = {} # 各sentenceの情報を入れる辞書
                            ans["sentence"] = original_sentence.replace('[AGENT]', x)
                            ans["Verb"] = column
                            ans["Agent"] = x
                            ans["Role"] = None
                            ans["Team"] = None
                            self.datalist.append(ans)
                    elif '[ROLE]' in original_sentence:
                        for y in Roles:
                            ans = {} # 各sentenceの情報を入れる辞書
                            ans["sentence"] = original_sentence.replace('[ROLE]', y)
                            ans["Verb"] = column
                            ans["Agent"] = None
                            ans["Role"] = y.upper()
                            ans["Team"] = None
                            self.datalist.append(ans)
        return self.datalist, len(self.datalist)
        
    def make_dataframe(self):
        self.make_data()
        df = pd.DataFrame(self.datalist)   
        df["Team"] = df["Team"].replace({"VILLAGER": "WHITE", "HUMAN": "WHITE", "WEREWOLF": "BLACK"})
        return df

    def dataframe_embed(self):
        df = self.make_dataframe()
        df["embedded"] = df["sentence"].map(lambda x: get_embedding(x))
        # リストに変換
        df["embedded"] = df["embedded"].map(lambda x: list(x))
        df.to_csv("embedded.csv")
        return df


In [169]:
a = MakeDataframe(Df)

In [170]:
a.make_data()

([{'sentence': 'I think Agent[00] is a villeger',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[00]',
   'Role': 'VILLEGER',
   'Team': None},
  {'sentence': 'I think Agent[00] is a seer',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[00]',
   'Role': 'SEER',
   'Team': None},
  {'sentence': 'I think Agent[00] is a werewolf',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[00]',
   'Role': 'WEREWOLF',
   'Team': None},
  {'sentence': 'I think Agent[00] is a possessed',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[00]',
   'Role': 'POSSESSED',
   'Team': None},
  {'sentence': 'I think Agent[01] is a villeger',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[01]',
   'Role': 'VILLEGER',
   'Team': None},
  {'sentence': 'I think Agent[01] is a seer',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[01]',
   'Role': 'SEER',
   'Team': None},
  {'sentence': 'I think Agent[01] is a werewolf',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[01]',
   'Role': 'WEREWOLF',
   'Team': None},
  {'sentence': 'I think Agent[01] is a po

In [171]:
a.make_dataframe()[1100:1150]

,sentence,Verb,Agent,Role,Team
1100,Everyone doesn't think Agent[02] is a villeger,None,Agent[02],VILLEGER,None
1101,Everyone doesn't think Agent[02] is a seer,None,Agent[02],SEER,None
1102,Everyone doesn't think Agent[02] is a werewolf,None,Agent[02],WEREWOLF,None
1103,Everyone doesn't think Agent[02] is a possessed,None,Agent[02],POSSESSED,None
1104,Everyone doesn't think Agent[03] is a villeger,None,Agent[03],VILLEGER,None
1105,Everyone doesn't think Agent[03] is a seer,None,Agent[03],SEER,None
1106,Everyone doesn't think Agent[03] is a werewolf,None,Agent[03],WEREWOLF,None
1107,Everyone doesn't think Agent[03] is a possessed,None,Agent[03],POSSESSED,None
1108,Everyone doesn't think Agent[04] is a villeger,None,Agent[04],VILLEGER,None
1109,Everyone doesn't think Agent[04] is a seer,None,Agent[04],SEER,None


In [172]:
a.dataframe_embed()

,sentence,Verb,Agent,Role,Team,embedded
0,I think Agent[00] is a villeger,ESTIMATE,Agent[00],VILLEGER,None,"[-0.034499943256378174, -0.006359028164297342,..."
1,I think Agent[00] is a seer,ESTIMATE,Agent[00],SEER,None,"[-0.02030394971370697, -0.011315640062093735, ..."
2,I think Agent[00] is a werewolf,ESTIMATE,Agent[00],WEREWOLF,None,"[-0.029915498569607735, -0.0012816088274121284..."
3,I think Agent[00] is a possessed,ESTIMATE,Agent[00],POSSESSED,None,"[-0.02937072515487671, 0.018204612657427788, -..."
4,I think Agent[01] is a villeger,ESTIMATE,Agent[01],VILLEGER,None,"[-0.03308890759944916, 0.0016154177719727159, ..."
...,...,...,...,...,...,...
1443,Everyone thinks what Agent[00] said is wrong,None,Agent[00],None,None,"[-0.010422752238810062, 0.002637328580021858, ..."
1444,Everyone thinks what Agent[01] said is wrong,None,Agent[01],None,None,"[-0.01157566998153925, -0.000851472606882453, ..."
1445,Everyone thinks what Agent[02] said is wrong,None,Agent[02],None,None,"[-0.004571947734802961, -0.0013156490167602897..."
1446,Everyone thinks what Agent[03] said is wrong,None,Agent[03],None,None,"[-0.008772888220846653, -0.0010623136768117547..."
